In [ ]:
import os
import re

import pandas
import requests

In [ ]:
data_path = os.path.join('.', 'data_as_provided')
output_path = os.path.join('.', 'data_processed')
arc_scenarios_file = os.path.join(data_path, 'ARC Employment Scenarios (45 sectors) v2.xlsx')
gb_baseline_file = os.path.join(data_path, 'LAD data by sectors.xlsx')

In [ ]:
baseline = pandas.read_excel(
    gb_baseline_file, 
    sheet_name=['GVA', 'Employment', 'Productivity'], 
    header=1, 
    index_col=0
)

In [ ]:
dfs = []
label_lookup = {
    'GVA': 'GVA (GBP2016m)',
    'Employment': 'Employment (000s)',
    'Productivity': 'Productivity (GBP2016 thousands per person in employment)'
}

def melt_block(df, label, year):
    df.index.name = "sector"
    df = df.reset_index().melt(
        id_vars=['sector'], var_name='lad_nm', value_name=label
    )
    df['year'] = year
    df.lad_nm = df.lad_nm.apply(lambda nm: nm.strip())
    df = df.set_index(
        ['year', 'lad_nm', 'sector']
    )
    return df

for label, df in baseline.items():
    # two blocks
    dfs.append(
        pandas.concat([
            melt_block(df[:45], label, 2018),
            melt_block(df[48:], label, 2050)
        ], axis=0)
    )
    
baseline_all = pandas.concat(dfs, axis=1, levels=['year','lad_nm', 'sector'])
baseline_all.head()

In [ ]:
# years * Great Britain LADs * sectors
assert len(baseline_all) == 2 * 380 * 45 

In [ ]:
variants = pandas.read_excel(
    arc_scenarios_file,
    sheet_name=['Baseline', 'Unplanned', 'City Focused', 'New Developments'],
    index_col=0,
    header=1
)

In [ ]:
df = variants['Baseline'].iloc[0:45, 0:29]
df.index.name = 'sector'
df = df.reset_index().melt(
    id_vars='sector', var_name='lad_nm', value_name='Employment')
assert len(df) == 45*29
df.head()

In [ ]:
def melt_scenario(s_df, scenario, name, year, row): 
    step = 45
    from_row, to_row = row, row + step
    df = s_df.iloc[from_row:to_row, 0:29].copy()
    df = df.reset_index().melt(
        id_vars=['sector'],
        var_name='lad_nm',
        value_name=name
    )
    df['year'] = year
    df['scenario'] = scenario
    df = df.set_index(
        ['scenario', 'year', 'lad_nm', 'sector']
    )
    return df

dfs = []
for scenario, s_df in variants.items():
    s_df.index.name = 'sector'
    s_dfs = [
        pandas.concat([
            melt_scenario(s_df, scenario, 'Employment', 2018, 0),
            melt_scenario(s_df, scenario, 'Employment', 2050, 49),
        ], axis=0),
        pandas.concat([
            melt_scenario(s_df, scenario, 'GVA', 2018, 98),
            melt_scenario(s_df, scenario, 'GVA', 2050, 147),
        ], axis=0),
        pandas.concat([
            melt_scenario(s_df, scenario, 'Productivity', 2018, 196),
            melt_scenario(s_df, scenario, 'Productivity', 2050, 245),
        ], axis=0)
    ]
    s_df_all = pandas.concat(s_dfs, axis=1, levels=['scenario', 'year', 'lad_nm', 'sector'])
    dfs.append(s_df_all)
        
variants_all = pandas.concat(dfs, axis=0)
variants_all.head()

In [ ]:
assert len(variants_all) == 2 * 29 * 4 * 45  # years * Arc LADs * scenarios * sectors

### Add LAD codes

In [ ]:
lad_nmcds = pandas.read_csv(os.path.join(data_path, 'lad_nmcd_changes.csv'))

In [ ]:
lad_nmcds = lad_nmcds[['lad11nm', 'lad11cd', 'lad16nm', 'lad16cd']]

In [ ]:
baseline_all_lad_nms = set(baseline_all.reset_index().lad_nm.unique())
all_lad_nms = set(lad_nmcds.lad11nm)

In [ ]:
all_lad_nms - baseline_all_lad_nms

In [ ]:
baseline_all_lad_nms - all_lad_nms

In [ ]:
baseline_all = baseline_all.reset_index()
baseline_all.lad_nm = baseline_all.lad_nm.replace({
    'Anglesey': 'Isle of Anglesey',
    'King`s Lynn and West Norfolk': "King's Lynn and West Norfolk",
    'Rhondda, Cynon, Taff': 'Rhondda Cynon Taf'
})

In [ ]:
baseline_wlad = baseline_all.merge(lad_nmcds, left_on='lad_nm', right_on='lad11nm').drop('lad_nm', axis=1)

In [ ]:
len(baseline_wlad.lad11nm.unique()), len(baseline_wlad), len(baseline_all)

In [ ]:
variants_wlad = variants_all.reset_index().merge(lad_nmcds, left_on='lad_nm', right_on='lad11nm').drop('lad_nm', axis=1)

In [ ]:
variants_wlad.head()

In [ ]:
len(variants_wlad.lad11nm.unique()), len(variants_wlad), len(variants_all)

In [ ]:
baseline_wlad[(baseline_wlad.lad11nm != baseline_wlad.lad16nm) | (baseline_wlad.lad11cd != baseline_wlad.lad16cd)].lad16nm.unique()

## Output data

In [ ]:
baseline_wlad.to_csv(os.path.join(output_path, 'gb_baseline.csv'), index=False)

In [ ]:
variants_wlad.to_csv(os.path.join(output_path, 'arc_variants.csv'), index=False)

### Merged, separate file-per-scenario

In [ ]:
base = baseline_wlad[
    ['year', 'sector', 'Employment', 'GVA', 'lad11nm', 'lad11cd', 'lad16nm', 'lad16cd']
].rename(columns={
    'Employment': 'employment', 
    'GVA': 'gva'
})
base = base[base.year.isin(range(2015, 2051))]
assert len(base) == 2 * 45 * 380  # years * sectors * LADs in GB

## Project Northern Ireland rGVA by LAD and sector

Look at projection for Northern Ireland based on 2015 industry composition, UK average growth

In [ ]:
def download(url, filename, force=False):
    if force or not os.path.exists(filename):
        r = requests.get(url, stream=True)
        with open(filename, 'wb') as fd:
            for chunk in r.iter_content(chunk_size=128):
                fd.write(chunk)

In [ ]:
# Nominal and real regional gross value added (balanced) by industry, NUTS1, NUTS2, NUTS3, 1998-2017
rgva_uk_ind_url = "https://www.ons.gov.uk/file?uri=/economy/grossvalueaddedgva/datasets/nominalandrealregionalgrossvalueaddedbalancedbyindustry/current/nominalandrealregionalgvabbyindustry.xlsx"
download(rgva_uk_ind_url, os.path.join(data_path, 'rgva_uk_industry.xlsx'))

In [ ]:
rgva = pandas.read_excel(
    os.path.join(data_path, 'rgva_uk_industry.xlsx'), 
    sheet_name='Table3c', 
    header=1)

In [ ]:
rgva = rgva[:13783]  # skip footnotes

In [ ]:
rgva.head()

In [ ]:
# pick division-level values (avoid double-count sectors, total)
p = re.compile('^\d')
rgva = rgva[rgva.SIC07.apply(lambda sic: bool(re.match(p, str(sic))))]

In [ ]:
lad_nuts3_lu_url = "http://geoportal1-ons.opendata.arcgis.com/datasets/e1e5de6c5fcc40c78adb03d84a2d299d_0.csv"
download(lad_nuts3_lu_url, os.path.join(data_path, "lad_nuts_lu.csv"))
lad_nuts = pandas.read_csv(os.path.join(data_path, 'lad_nuts_lu.csv'))
lad_nuts = lad_nuts[['LAD16CD', 'LAD16NM', 'NUTS318CD']].sort_values('LAD16CD').drop_duplicates().rename(columns={
    'LAD16CD': 'lad16cd',
    'LAD16NM': 'lad16nm',
    'NUTS318CD': 'nuts318cd'
})
lad_nuts.head(), len(lad_nuts.nuts318cd.unique()), len(lad_nuts.lad16cd.unique())

In [ ]:
rgva15 = rgva.merge(
    lad_nuts, left_on='Region code', right_on='nuts318cd', how='outer'
).rename(columns={
    'Region name': 'nuts318nm',
    2015: 'gva15_nuts_division_group',
    'SIC07': 'sic07_division_group',
    'SIC07 description': 'sic07_division_group_description'
})[[
    'lad16cd',
    'lad16nm',
    'nuts318cd',
    'nuts318nm',
    'sic07_division_group',
    'sic07_division_group_description',
    'gva15_nuts_division_group'
]]
rgva15.sic07_division_group = rgva15.sic07_division_group.astype(str)
rgva15

In [ ]:
sector_map = pandas.read_csv(os.path.join('data_as_provided','map_sectors.csv'))
sector_map

In [ ]:
rgva15.sic07_division_group = rgva15.sic07_division_group.apply(lambda d: d.replace('-', ' to '))

In [ ]:
rgva15s = rgva15.merge(sector_map, on='sic07_division_group', how='outer')
rgva15s

### Disaggregate UK 2015 rGVA to LADs and SIC07 divisions

Very coarse, purely proportional split - could be improved e.g. by using BRES employment industry percentage figures for current split to full divisions

In [ ]:
count_for_disagg = rgva15s.groupby(['nuts318cd', 'sic07_division_group']).count()[['lad16cd']]
count_for_disagg.columns = ['nuts_division_group_count']
count_for_disagg = count_for_disagg.reset_index()
rgva15sd = rgva15s.merge(count_for_disagg, on=['nuts318cd', 'sic07_division_group'], how='left')
rgva15sd

In [ ]:
rgva15sd['gva15_lad_division'] = rgva15sd.gva15_nuts_division_group / rgva15sd.nuts_division_group_count
rgva15_lad_division = rgva15sd[[
    'lad16cd', 'itrc_sector', 'sic07_division', 'gva15_lad_division'
]]
rgva15_lad_division.head()

In [ ]:
sum_for_disagg = rgva15_lad_division.groupby(['lad16cd']).sum()[['gva15_lad_division']]
sum_for_disagg.columns = ['gva15_lad_total']
rgva15_lad_division_d = rgva15_lad_division.merge(sum_for_disagg.reset_index(), on='lad16cd')
rgva15_lad_division_d['gva15_lad_division_proportion'] = rgva15_lad_division_d.gva15_lad_division / rgva15_lad_division_d.gva15_lad_total
rgva15_lad_division_d.head()

### Project NI regions as proportion of NI total

Where future NI total is projected assuming NI growth equals GB growth:

NI GVA in 2015 * (GB GVA in future year / GB GVA in 2015) = NI GVA in future year

And NI future regional/sectoral GVA follows the same proportional structure as in 2015.

In [ ]:
ni_rgva = rgva15_lad_division_d[rgva15_lad_division_d.lad16cd.str.startswith('N')].copy()
ni_rgva.sort_values(['lad16cd', 'sic07_division']).head()

In [ ]:
ni_rgva_for_proj = ni_rgva[['lad16cd', 'gva15_lad_total']].drop_duplicates()
ni_rgva_for_proj['gva15_lad_ni_proportion'] = ni_rgva_for_proj.gva15_lad_total / ni_rgva_for_proj.gva15_lad_total.sum()
ni_rgva_for_proj

In [ ]:
gb_base = base.copy()[['year', 'lad16cd', 'gva', 'employment']]
gb_base.head()

In [ ]:
gb_growth = gb_base.groupby('year').sum()[['gva']].reset_index()
gb_growth.head()

In [ ]:
total_ni_rgva15 = ni_rgva_for_proj.gva15_lad_total.sum()
total_uk_rgva15 = rgva15_lad_division_d.gva15_lad_division.sum()
total_gb_rgva15 = total_uk_rgva15 - total_ni_rgva15

In [ ]:
dfs = []
for year in gb_growth.year:
    df = ni_rgva_for_proj.copy()
    df['year'] = year
    gb_future = float(gb_growth[gb_growth.year == year].gva)
    ni_future = total_ni_rgva15 * (gb_future / total_gb_rgva15)
    df['gva'] = df.gva15_lad_ni_proportion * ni_future
    dfs.append(df)
    
ni_base = pandas.concat(dfs, axis=0)[['year', 'lad16cd', 'gva']]
ni_base

### Disaggregate projections by SIC07 division, reaggregate to ITRC sector

Assuming constant sectoral shares of GVA, projected LAD sectoral GVA is (projected LAD GVA * (current LAD sectoral GVA / current LAD GVA).

In [ ]:
ni_disagg = ni_base.merge(rgva15_lad_division_d, on='lad16cd', how='left').rename(columns={'year': 'timestep'})
ni_disagg.head()

In [ ]:
ni_disagg['gva_lad_division'] = ni_disagg.gva * ni_disagg.gva15_lad_division_proportion

In [ ]:
ni_disagg = ni_disagg.groupby(
    ['timestep', 'lad16cd', 'itrc_sector']
).sum().reset_index()[
    ['timestep', 'lad16cd', 'itrc_sector', 'gva_lad_division']
].dropna().rename(
    columns={'gva_lad_division': 'gva_per_sector'}
)
ni_disagg = ni_disagg[ni_disagg.itrc_sector != 46]  # drop unallocated/household

In [ ]:
assert len(ni_disagg) == 2 * 11 * 45  # timestep * NI LADs * 45 sectors
ni_disagg['employment'] = 0  # no estimate for NI employment
ni_disagg.sort_values(by=['timestep', 'lad16cd', 'itrc_sector']).tail()

In [ ]:
base.head()

In [ ]:
# add itrc sector codes to GB base
base.sector = base.sector.apply(lambda d: d.replace(', etc', ' etc'))
sector_ids = sector_map[['itrc_sector', 'itrc_sector_description']].drop_duplicates()
sector_ids.itrc_sector_description = sector_ids.itrc_sector_description.apply(lambda d: d.replace(', etc', ' etc'))
gb_disagg = base.merge(
    sector_ids, left_on='sector', right_on='itrc_sector_description', how='left'
)[
    ['year', 'lad16cd', 'itrc_sector', 'gva', 'employment']
].rename(
    columns={'year': 'timestep', 'gva': 'gva_per_sector'}
)
assert len(gb_disagg) == 2 * 380 * 45
gb_disagg.head()

In [ ]:
full_base = pandas.concat([gb_disagg, ni_disagg], axis=0)
assert len(full_base) == 391 * 45 * 2
full_base.to_csv(os.path.join(output_path, 'uk_baseline.csv'), index=False)

### Stitch scenario projections with baseline UK (GVA by LAD)

In [ ]:
variants_wlad = variants_wlad.rename(columns={'lad16cd': 'lad_uk_2016'})
full_base = full_base.rename(columns={'lad16cd': 'lad_uk_2016'})

In [ ]:
base_gva = full_base[['timestep', 'lad_uk_2016', 'itrc_sector', 'gva_per_sector']].copy()
base_emp = full_base[['timestep', 'lad_uk_2016', 'itrc_sector', 'employment']].copy()

In [ ]:
# Baseline 2018/2050 for all non-Arc LADs
base_gva = base_gva[~base_gva.lad_uk_2016.isin(variants_wlad.lad_uk_2016.unique())]
assert len(base_gva) == (391 - 29) * 45 * 2
base_emp = base_emp[~base_emp.lad_uk_2016.isin(variants_wlad.lad_uk_2016.unique())]
assert len(base_emp) == (391 - 29) * 45 * 2
base_gva.head()

In [ ]:
# add itrc sector codes to GB base
variants_wlad.sector = variants_wlad.sector.apply(lambda d: d.replace(', etc', ' etc'))
variants_wlad = variants_wlad.merge(
    sector_ids, left_on='sector', right_on='itrc_sector_description', how='left'
)[
    ['year', 'lad_uk_2016', 'itrc_sector', 'GVA', 'Employment', 'scenario']
].rename(
    columns={'year': 'timestep', 'GVA': 'gva_per_sector', 'Employment': 'employment'}
)

In [ ]:
variants_wlad.head()

In [ ]:
def interpolate(data, lower_year, upper_year, projected_year):
    proportion = (projected_year - lower_year) / (upper_year - lower_year)
    step = proportion * (data[upper_year] - data[lower_year])
    return data[lower_year] + step

In [ ]:
def interpolate_region_sector_df(df, value_colname, timesteps):
    df["rs"] = df.apply(lambda d: (d.lad_uk_2016, d.itrc_sector), axis=1) # hack index as tuple
    df = df.pivot(index="rs", columns="timestep", values=value_colname)
    for year in timeline:
        df[year] = interpolate(df, 2018, 2050, year)
    df = df.reset_index()
    df['itrc_sector'] = df.rs.apply(lambda rs: rs[1])
    df['lad_uk_2016'] = df.rs.apply(lambda rs: rs[0])
    df.drop("rs", axis=1, inplace=True)
    return df.melt(id_vars=['itrc_sector', 'lad_uk_2016'], value_name=value_colname)

def interpolate_region_df(df, value_colname, timesteps):
    df = df.pivot(index="lad_uk_2016", columns="timestep", values=value_colname)
    for year in timeline:
        df[year] = interpolate(df, 2018, 2050, year)
    df = df.reset_index()
    return df.melt(id_vars=['itrc_sector', 'lad_uk_2016'], value_name=value_colname)

In [ ]:
timeline = list(range(2015, 2051))

In [ ]:
# Set up variants dict
vard = {}

for scenario in ('Baseline', 'Unplanned', 'City Focused', 'New Developments'):
    var = variants_wlad[
        variants_wlad.scenario == scenario
    ][
        ['timestep', 'employment', 'gva_per_sector', 'lad_uk_2016', 'itrc_sector']
    ]
    vard[scenario] = var.copy()

In [ ]:
# Double check baseline 'scenario' matches full baseline
arc_baseline_from_scenario = vard['Baseline'][
    ['timestep','lad_uk_2016','itrc_sector','gva_per_sector','employment']
].sort_values(['timestep','lad_uk_2016','itrc_sector']).reset_index(drop=True)

arc_baseline_from_gb = full_base[full_base.lad_uk_2016.isin(variants_wlad.lad_uk_2016.unique())][
    ['timestep','lad_uk_2016','itrc_sector','gva_per_sector','employment']
].sort_values(['timestep','lad_uk_2016','itrc_sector']).reset_index(drop=True)

import pandas.testing
pandas.testing.assert_frame_equal(arc_baseline_from_scenario, arc_baseline_from_gb)

In [ ]:
def output_for_smif(df, var_name, scenario_key):
    # full, by lad/sector
    df.to_csv(os.path.join(output_path, 'arc_{}_by_sector__{}.csv'.format(var_name, scenario_key)), index=False)
    
    # by lad / energy demand sector    
    energy_demand_sector_subset = [2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 14, 15, 17, 19, 23, 27, 28, 29, 35, 40, 41]
    df_ed = df[df.itrc_sector.isin(energy_demand_sector_subset)]
    df_ed.to_csv(os.path.join(output_path, 'arc_{}_by_sector_for_energy_demand__{}.csv'.format(var_name, scenario_key)), index=False)
    
    # by lad (drop sectors)
    df_by_lad = df.drop('itrc_sector', axis=1).groupby(['timestep', 'lad_uk_2016']).sum().reset_index() \
        .rename(columns={'gva_per_sector': 'gva'})
    df_by_lad.to_csv(os.path.join(output_path, 'arc_{}__{}.csv'.format(var_name, scenario_key)), index=False)

In [ ]:
for scenario, key in (('Baseline', 'baseline'), ('Unplanned', '0-unplanned'), ('New Developments', '1-new-cities'), ('City Focused', '2-expansion')):
    # split / stitch
    scenario_emp = pandas.concat(
        [base_emp, vard[scenario][['timestep', 'lad_uk_2016', 'itrc_sector', 'employment']]], axis=0, sort=False
    ).sort_values(
        ['timestep', 'lad_uk_2016', 'itrc_sector']
    )
    scenario_gva = pandas.concat(
        [base_gva, vard[scenario][['timestep', 'lad_uk_2016', 'itrc_sector', 'gva_per_sector']]], axis=0, sort=False
    ).sort_values(
        ['timestep', 'lad_uk_2016', 'itrc_sector']
    )
    
    # Interpolate
    scenario_emp = interpolate_region_sector_df(scenario_emp, 'employment', timeline)
    scenario_gva = interpolate_region_sector_df(scenario_gva, 'gva_per_sector', timeline)
    
    # Output to CSV
    output_for_smif(scenario_emp, 'employment', key)
    output_for_smif(scenario_gva, 'gva', key)